# Two-View 3D Reconstruction

This notebook implements a simple two-view Structure from Motion approach to create a 3D reconstruction from two images (01.jpg and 02.jpg).

## Setup and Requirements

### Theory Introduction

Structure from Motion (SfM) is a photogrammetric range imaging technique for estimating three-dimensional structures from two-dimensional image sequences. It works by:

1. Finding correspondences between images
2. Recovering camera poses (position and orientation)
3. Triangulating points to create a 3D reconstruction

This notebook demonstrates the classic two-view reconstruction which forms the foundation of more complex multi-view reconstruction systems.

In [ ]:
import sys
import os
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
  if os.path.exists('/content/Prague_ml_data'):
    print("Prague_ml_data already exists")
  else:
    ! git clone https://github.com/VarunBurde/Prague_ml_data

### Environment Setup

The code above checks if we're running in Google Colab and clones the necessary data repository if needed. This ensures we have access to the sample images and calibration data regardless of where this notebook is executed.

In [ ]:
# Install required packages
!pip install open3d networkx matplotlib tqdm opencv-python

### Required Libraries

- **Open3D**: A library for 3D data processing and visualization
- **NetworkX**: A library for network analysis (used for some graph-based operations)
- **Matplotlib**: For 2D plotting and visualization
- **tqdm**: For progress bars
- **OpenCV**: Core library for computer vision algorithms, including feature detection and matching

In [ ]:
import cv2
import open3d as o3d
import os
import numpy as np
from matplotlib import pyplot as plt
from glob import glob
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

# For displaying Open3D visualizations in notebooks
from google.colab.patches import cv2_imshow
import base64
from IPython.display import HTML

def display_open3d_to_notebook(vis, width=900, height=600):
    vis.update_renderer()
    image = vis.capture_screen_float_buffer()
    image_array = np.asarray(image)
    image_array = (image_array * 255).astype(np.uint8)
    image_array = cv2.cvtColor(image_array, cv2.COLOR_RGB2BGR)
    _, encoded_img = cv2.imencode('.png', image_array)
    encoded_img = base64.b64encode(encoded_img)
    html = f'<img src="data:image/png;base64,{encoded_img.decode()}" width="{width}" height="{height}"/>'
    return HTML(html)

### Helper Functions for Visualization

The `display_open3d_to_notebook` function enables visualization of Open3D renderings in the notebook environment. It works by:

1. Capturing the rendered Open3D visualization as an image buffer
2. Converting the buffer to a numpy array and applying necessary color transformations
3. Encoding the image as base64 and embedding it in HTML for display in the notebook

This is particularly useful for interactive 3D visualization in environments like Google Colab.

## 1. Data Loading

First, let's load our two specific images: 01.jpg and 02.jpg.

### Image Data for Structure from Motion

For two-view reconstruction, we need:

1. Two images of the same scene from different viewpoints
2. Camera calibration information (intrinsic matrix K)

The intrinsic matrix K contains information about the camera's internal parameters:
- Focal length (fx, fy)
- Principal point (cx, cy)
- Skew coefficient

This information is essential for accurate 3D reconstruction as it allows us to convert between pixel coordinates and normalized camera coordinates.

In [ ]:
# For Colab: Upload images
from google.colab import files
import shutil

# Set the paths for our specific images
if IN_COLAB:
  img_path = "/content/Prague_ml_data/dataset/two_view_data/images"
  image_paths = [os.path.join(img_path, "01.jpg"), os.path.join(img_path, "02.jpg")]
  K = np.loadtxt("/content/Prague_ml_data/dataset/two_view_data/K.txt")
else:
  img_path = "dataset/two_view_data/images"
  image_paths = [os.path.join(img_path, "01.jpg"), os.path.join(img_path, "02.jpg")]
  K = np.loadtxt("dataset/two_view_data/K.txt")

plt.figure(figsize=(15, 7))
for i, path in enumerate(image_paths):
    plt.subplot(1, 2, i+1)
    img = cv2.imread(path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.title(f"Image {i+1}: {os.path.basename(path)}")
plt.tight_layout()
plt.show()

## 2. Load Images and Extract Features

Now we'll load our two images and extract feature points using SIFT.

### Feature Detection Theory

Feature detection is a crucial first step in 3D reconstruction. We need to identify distinctive points in each image that can be reliably matched between views.

**SIFT (Scale-Invariant Feature Transform)**:

SIFT is a robust feature detection algorithm that identifies keypoints which are invariant to:
- Scale changes
- Rotation
- Illumination changes
- Viewpoint changes (to some extent)

The SIFT algorithm works by:
1. Building a scale-space pyramid of the image
2. Finding extrema (maxima/minima) in the difference of Gaussian images
3. Refining keypoint locations
4. Eliminating low-contrast and edge keypoints
5. Assigning orientation to each keypoint
6. Creating descriptors (128-dimensional vectors) for each keypoint

These descriptors capture the local appearance of the image around each keypoint and are used for matching between images.

In [ ]:
# Load our two images
images = []
images_rgb = []

for path in image_paths:
    img = cv2.imread(path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    images.append(img)
    images_rgb.append(img_rgb)

print(f"Loaded {len(images)} images")

# Create SIFT detector with more aggressive parameters for denser features
sift = cv2.SIFT_create(
    nfeatures=0,  # No limit on number of features
    contrastThreshold=0.04,  # Lower threshold for more features
    edgeThreshold=10,  # Higher threshold for more features
    sigma=1.6  # Default sigma
)

### SIFT Parameter Tuning

The SIFT detector is created with specific parameters to optimize feature detection:

- **nfeatures=0**: No limit on the number of features to detect (default is 0, which means no limit)
- **contrastThreshold=0.04**: Lower values detect more features but may include less distinctive ones
- **edgeThreshold=10**: Higher values allow more features near edges
- **sigma=1.6**: Initial Gaussian blur applied to the image

Tuning these parameters involves balancing quantity (more features) with quality (more distinctive features). For 3D reconstruction, we generally want a dense set of reliable features.

In [ ]:
# Detect features in both images
all_kp = []
all_des = []

for i, img_rgb in enumerate(images_rgb):
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    img_eq = cv2.equalizeHist(img_gray)
    
    # Detect features at multiple scales
    kp_standard, des_standard = sift.detectAndCompute(img_rgb, None)
    kp_eq, des_eq = sift.detectAndCompute(img_eq, None)
    
    # Combine features
    if des_standard is not None and des_eq is not None:
        kp = kp_standard + kp_eq
        des = np.vstack((des_standard, des_eq))
    elif des_standard is not None:
        kp = kp_standard
        des = des_standard
    elif des_eq is not None:
        kp = kp_eq
        des = des_eq
    else:
        kp = []
        des = np.array([])
    
    all_kp.append(kp)
    all_des.append(des)
    print(f"Image {i+1} ({os.path.basename(image_paths[i])}): Detected {len(kp)} keypoints")

### Enhanced Feature Detection

The code above implements a robust feature detection approach by:

1. **Multi-scale processing**: Detecting features in both RGB and histogram-equalized grayscale versions of the image

2. **Histogram equalization**: This technique improves contrast in the image, which can reveal features in darker or low-contrast regions

3. **Feature combination**: Combining features from both versions increases the overall feature density

This approach helps ensure we have a rich set of features throughout the image, which is essential for a complete 3D reconstruction.

### Visualize Detected Features

Let's visualize the keypoints detected in both images.

In [ ]:
# Display keypoints on both images
plt.figure(figsize=(15, 7))
for i in range(2):
    plt.subplot(1, 2, i+1)
    img_with_kp = cv2.drawKeypoints(images_rgb[i], all_kp[i], None, 
                                    flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    plt.imshow(img_with_kp)
    plt.title(f"Image {i+1} ({os.path.basename(image_paths[i])}): {len(all_kp[i])} keypoints")
plt.tight_layout()
plt.show()

### Feature Visualization

The visualization shows the detected SIFT keypoints on each image. Each keypoint is represented by a circle:
- The center indicates the keypoint location
- The radius indicates the keypoint scale
- The line indicates the keypoint orientation

A good feature distribution should cover the image uniformly, with concentration in areas with distinctive texture or structures. The size of the circles indicates the scale at which the feature was detected - larger circles represent features detected at coarser scales.

## 3. Match Features Between Images

Now we'll match features between our two images to find correspondences.

### Feature Matching Theory

Feature matching is the process of finding correspondences between keypoints in different images. This is a critical step in 3D reconstruction as these correspondences will be used to establish geometric relationships between views.

**FLANN-based Matching**:

Fast Library for Approximate Nearest Neighbors (FLANN) is an efficient algorithm for finding approximate nearest neighbors in high-dimensional spaces. For feature matching:

1. **KD-Tree Algorithm**: Organizes descriptors in a tree-like structure for efficient searching

2. **K-Nearest Neighbor (KNN) Matching**: For each keypoint in the first image, find the two closest matches in the second image

3. **Lowe's Ratio Test**: Filter matches by comparing the distances between the best and second-best match
   - If best_match_distance < ratio * second_best_match_distance, keep the match
   - This helps ensure that matches are distinctive and reduces false positives

4. **Cross-Checking**: An additional validation step that ensures matches are consistent in both directions
   - A match is valid only if keypoint A in image 1 matches to keypoint B in image 2
   - AND keypoint B in image 2 matches back to keypoint A in image 1

In [ ]:
# Improved FLANN matcher setup
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=100)  # More checks for better accuracy
flann = cv2.FlannBasedMatcher(index_params, search_params)

# Function to perform feature matching with cross-check
def match_features(des1, des2, lowes_ratio=0.75, cross_check=True):
    # Forward matching (img1 -> img2)
    if des1 is None or des2 is None or len(des1) == 0 or len(des2) == 0:
        return []
    
    matches12 = flann.knnMatch(des1, des2, k=2)
    good_matches12 = []
    for m, n in matches12:
        if m.distance < lowes_ratio * n.distance:
            good_matches12.append(m)
    
    if not cross_check:
        return good_matches12
    
    # Backward matching (img2 -> img1) for cross-check
    matches21 = flann.knnMatch(des2, des1, k=2)
    good_matches21 = []
    for m, n in matches21:
        if m.distance < lowes_ratio * n.distance:
            good_matches21.append(m)
    
    # Cross-check: keep matches that are consistent in both directions
    cross_matches = []
    for match12 in good_matches12:
        for match21 in good_matches21:
            # Check if match is consistent in both directions
            if match12.queryIdx == match21.trainIdx and match12.trainIdx == match21.queryIdx:
                cross_matches.append(match12)
                break
    
    return cross_matches

# Get matched points from keypoints and matches
def get_matched_points(kp1, kp2, matches):
    pts1 = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    pts2 = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    return pts1, pts2

### Robust Feature Matching Implementation

The code implements a robust feature matching approach with several key components:

1. **FLANN Matcher Configuration**:
   - Uses KD-Tree algorithm with 5 trees for efficient searching
   - Sets checks=100 to increase matching accuracy (more computation but better results)

2. **Lowe's Ratio Test**:
   - The default ratio is 0.75 (a common value in practice)
   - Lower values make matching more strict, higher values allow more matches but may include more false positives

3. **Cross-Check Validation**:
   - Performs matching in both directions (image1→image2 and image2→image1)
   - Only keeps matches that are mutually consistent
   - This significantly improves match quality at the cost of reducing match quantity

4. **Helper Function for Extracting Point Coordinates**:
   - Converts OpenCV match objects into actual point coordinates
   - The resulting coordinates will be used for geometric calculations in the next steps

In [ ]:
# Match features between our two images (01.jpg and 02.jpg)
idx1, idx2 = 0, 1  # Using image indices 0 and 1 (01.jpg and 02.jpg)
good_matches = match_features(all_des[idx1], all_des[idx2], lowes_ratio=0.75)


### Applying Feature Matching

Here we apply the feature matching between our two images using the robust matching function defined earlier. The function:

1. Takes feature descriptors from both images
2. Applies FLANN-based KNN matching in both directions
3. Filters matches using Lowe's ratio test with a threshold of 0.75
4. Applies cross-checking to ensure matches are consistent

The result is a set of high-quality matches that will form the basis for our geometric calculations in the following steps.

In [ ]:
print(f"Found {len(good_matches)} good matches between {os.path.basename(image_paths[0])} and {os.path.basename(image_paths[1])}")


img1 = images_rgb[idx1]
img2 = images_rgb[idx2]
h1, w1 = img1.shape[:2]
h2, w2 = img2.shape[:2]
matched_img = np.zeros((max(h1, h2), w1 + w2, 3), dtype=np.uint8)
matched_img[:h1, :w1] = img1
matched_img[:h2, w1:w1 + w2] = img2

# Draw keypoints
for m in good_matches[:10]:
    pt1 = tuple(map(int, all_kp[idx1][m.queryIdx].pt))
    pt2 = tuple(map(int, all_kp[idx2][m.trainIdx].pt))
    pt2 = (int(pt2[0] + w1), int(pt2[1]))  # shift pt2 x-coord

    # Draw circles
    cv2.circle(matched_img, pt1, 20, (0, 255, 0), -1)  # radius 6, green
    cv2.circle(matched_img, pt2, 20, (0, 255, 0), -1)

    # Draw connecting line
    cv2.line(matched_img, pt1, pt2, (255, 0, 0),10)  # blue line, thickness 2

plt.figure(figsize=(15, 10))
plt.imshow(matched_img)
plt.title(f'Matches between {os.path.basename(image_paths[0])} and {os.path.basename(image_paths[1])}')
plt.show()

### Match Visualization

The visualization shows the top 10 feature matches between the two images. Good matches should have the following characteristics:

1. **Consistent Motion**: The connecting lines should generally follow similar directions (indicating consistent camera motion)

2. **Spatial Distribution**: Matches should be distributed across the images (not concentrated in one area)

3. **Low Outlier Rate**: There should be few obvious mismatches (lines connecting unrelated points)

This visual inspection helps confirm the quality of feature matching before proceeding to the geometric calculations.

## 4. Two-View Reconstruction

Now we'll use the matched features to perform 3D reconstruction.

### Epipolar Geometry Theory

Two-view reconstruction is based on epipolar geometry, which describes the geometric relationship between two camera views:

**Key Concepts**:

1. **Essential Matrix (E)**: Encodes the geometric relationship between two calibrated cameras
   - E = [t]₊R (where [t]₊ is the skew-symmetric matrix of the translation vector, and R is the rotation matrix)
   - For corresponding points x and x', we have: x'ᵀEx = 0 (epipolar constraint)

2. **Camera Pose Recovery**:
   - From the essential matrix, we can extract the relative rotation (R) and translation (t) between cameras
   - By convention, the first camera is at the origin (identity rotation, zero translation)
   - The second camera's pose is defined relative to the first

3. **Triangulation**:
   - Once we know both camera poses and have matched points, we can triangulate 3D points
   - Each pair of matching 2D points defines rays in 3D space
   - The intersection of these rays (or closest point to both rays) gives us the 3D point

In [ ]:
# Initialize camera poses (R|t) for each image
# First camera (01.jpg) is at origin with identity rotation
camera_poses = [np.hstack((np.eye(3), np.zeros((3, 1))))]
camera_matrices = [K @ camera_poses[0]]

# Structure to store 3D points and their 2D observations
point_cloud = []
point_colors = []

pts1, pts2 = get_matched_points(all_kp[idx1], all_kp[idx2], good_matches)

# Calculate essential matrix with robust parameters
E, mask = cv2.findEssentialMat(pts1, pts2, K, method=cv2.RANSAC, prob=0.999, threshold=2.0)

# Recover pose for second camera (02.jpg)
_, R, t, mask = cv2.recoverPose(E, pts1, pts2, K, mask=mask)

# Set second camera pose
camera_poses.append(np.hstack((R, t)))
camera_matrices.append(K @ camera_poses[1])

print(f"Camera 1 (01.jpg) pose:\n{camera_poses[0]}")
print(f"\nCamera 2 (02.jpg) pose:\n{camera_poses[1]}")

# Filter points using mask
mask = mask.ravel() == 1
pts1_good = pts1[mask]
pts2_good = pts2[mask]

print(f"\nUsing {np.sum(mask)} inlier matches for triangulation")

### Camera Pose Estimation

This section implements the first major step in 3D reconstruction: estimating the relative poses of the two cameras. The process includes:

1. **Camera Initialization**:
   - The first camera (01.jpg) is defined as the world origin with identity rotation and zero translation
   - This establishes our coordinate system reference frame

2. **Essential Matrix Estimation**:
   - Uses `cv2.findEssentialMat` with RANSAC to robustly estimate the essential matrix
   - The RANSAC method helps eliminate outlier correspondences
   - Parameters include:
     - `prob=0.999`: High probability of finding the correct model
     - `threshold=2.0`: Error threshold for inlier classification (in pixels)

3. **Pose Recovery**:
   - Uses `cv2.recoverPose` to extract rotation (R) and translation (t) from the essential matrix
   - This gives us the relative pose of the second camera
   - The function also returns a refined mask indicating which points are geometrically consistent

4. **Point Filtering**:
   - Only points that are inliers to the essential matrix are used for triangulation
   - This further improves reconstruction quality by eliminating outliers

In [ ]:
# Triangulate points
pts1_good = pts1_good.reshape(-1, 2).T
pts2_good = pts2_good.reshape(-1, 2).T
points_4D = cv2.triangulatePoints(camera_matrices[0], camera_matrices[1], pts1_good, pts2_good)
points_3D = points_4D / points_4D[3]  # Convert to Cartesian
points_3D = points_3D[:3, :].T

# Filter points by depth
valid_points = []
valid_colors = []

for i in range(points_3D.shape[0]):
    point = points_3D[i]
    # Check if point is in front of both cameras
    if point[2] > 0:
        # Get corresponding 2D points
        x1, y1 = pts1_good[:, i]
        
        # Extract color from first image
        x1_int, y1_int = int(round(x1)), int(round(y1))
        if 0 <= x1_int < images_rgb[idx1].shape[1] and 0 <= y1_int < images_rgb[idx1].shape[0]:
            color = images_rgb[idx1][y1_int, x1_int] / 255.0
            valid_points.append(point)
            valid_colors.append(color)

point_cloud = np.array(valid_points)
point_colors = np.array(valid_colors)

print(f"Reconstructed point cloud has {len(point_cloud)} points")

### 3D Triangulation

Triangulation is the process of determining the 3D position of points from their 2D projections in multiple images. This code implements triangulation with the following steps:

1. **Triangulation Algorithm**:
   - `cv2.triangulatePoints` computes 3D points in homogeneous coordinates (4D)
   - It uses Direct Linear Transform (DLT) algorithm to solve the triangulation problem
   - Inputs are the camera projection matrices and corresponding 2D points

2. **Homogeneous to Cartesian Conversion**:
   - Divides the 4D homogeneous coordinates by the fourth component (w)
   - Takes the first three components (x, y, z) as the Cartesian coordinates

3. **Point Filtering**:
   - Cheirality check: Only keeps points with positive z-coordinate (in front of both cameras)
   - This eliminates points that are behind either camera (physically impossible)

4. **Color Assignment**:
   - Assigns colors to 3D points based on their color in the first image
   - This allows for a visually meaningful 3D reconstruction

The resulting point cloud represents the 3D structure of the scene visible in both images.

### Visualize Point Cloud
Let's visualize the 3D point cloud reconstructed from our two images.

In [ ]:
# Create Open3D point cloud for visualization
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud)
pcd.colors = o3d.utility.Vector3dVector(point_colors)

# Remove outliers for cleaner visualization
pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

### Point Cloud Refinement

The final point cloud is prepared for visualization with some additional processing:

1. **Open3D Point Cloud Creation**:
   - Converts numpy arrays to Open3D's Vector3dVector format
   - Assigns both 3D positions and colors to the point cloud

2. **Statistical Outlier Removal**:
   - Uses Open3D's statistical outlier removal algorithm to clean the point cloud
   - Parameters:
     - `nb_neighbors=20`: Uses 20 nearest neighbors to evaluate each point
     - `std_ratio=2.0`: Points with average distance larger than 2 standard deviations are removed
   - This helps remove isolated points that are likely reconstruction errors

A clean point cloud is important for both visualization quality and subsequent processing steps if the reconstruction were to be further refined.

In [ ]:
def draw_geometries(geometries):
    graph_objects = []

    for geometry in geometries:
        geometry_type = geometry.get_geometry_type()

        if geometry_type == o3d.geometry.Geometry.Type.PointCloud:
            points = np.asarray(geometry.points)
            colors = None
            if geometry.has_colors():
                colors = np.asarray(geometry.colors)
            elif geometry.has_normals():
                colors = (0.5, 0.5, 0.5) + np.asarray(geometry.normals) * 0.5
            else:
                geometry.paint_uniform_color((1.0, 0.0, 0.0))
                colors = np.asarray(geometry.colors)

            scatter_3d = go.Scatter3d(x=points[:,0], y=points[:,1], z=points[:,2], mode='markers', marker=dict(size=1, color=colors))
            graph_objects.append(scatter_3d)

        if geometry_type == o3d.geometry.Geometry.Type.TriangleMesh:
            triangles = np.asarray(geometry.triangles)
            vertices = np.asarray(geometry.vertices)
            colors = None
            if geometry.has_triangle_normals():
                colors = (0.5, 0.5, 0.5) + np.asarray(geometry.triangle_normals) * 0.5
                colors = tuple(map(tuple, colors))
            else:
                colors = (1.0, 0.0, 0.0)

            mesh_3d = go.Mesh3d(x=vertices[:,0], y=vertices[:,1], z=vertices[:,2], i=triangles[:,0], j=triangles[:,1], k=triangles[:,2], facecolor=colors, opacity=0.50)
            graph_objects.append(mesh_3d)

    fig = go.Figure(
        data=graph_objects,
        layout=dict(
            scene=dict(
                xaxis=dict(visible=False),
                yaxis=dict(visible=False),
                zaxis=dict(visible=False)
            )
        )
    )
    fig.show()

### Interactive 3D Visualization

This helper function enables interactive 3D visualization of the point cloud using Plotly:

1. **Geometry Processing**:
   - Handles different types of Open3D geometries (PointCloud, TriangleMesh)
   - Extracts points, vertices, triangles, and colors from the geometry objects

2. **Plotly Integration**:
   - Creates appropriate Plotly 3D visualization objects (Scatter3d for points, Mesh3d for meshes)
   - Configures the layout for clean visualization without axis clutter

3. **Visualization Features**:
   - Points are rendered as small markers with their original colors
   - Meshes would be rendered with semi-transparency for better visualization
   - The resulting visualization is interactive, allowing rotation, zoom, and pan

In [ ]:
o3d.visualization.draw_geometries = draw_geometries # replace function
o3d.visualization.draw_geometries([pcd])

### Final 3D Point Cloud Visualization

The final visualization displays the reconstructed 3D point cloud in an interactive viewer. This allows for:

1. **Inspection of Reconstruction Quality**:
   - Examining the density and distribution of reconstructed points
   - Checking for structural coherence and recognizable scene elements

2. **Visualization Benefits**:
   - Interactive rotation helps understand the true 3D nature of the reconstruction
   - Color information aids in identifying scene components

3. **Potential Next Steps**:
   - This point cloud could be further processed for mesh reconstruction
   - Additional views could be incorporated for a denser, more complete reconstruction
   - Bundle adjustment could refine camera poses and 3D point positions

The two-view reconstruction demonstrates the fundamental principles of Structure from Motion, which form the basis for more complex multi-view reconstruction systems used in applications like 3D scanning, AR/VR content creation, and robotics.